# Давайте создадим AI чат бота...

## Идея

#### Давайте обсудим, для чего это нужно

1. Сильный AI может стать бесполезным, если не сможет общаться с людьми
2. Даже если не мечтать про сильный AI, умение "понимать" и осмысленно отвечать на человеческом языке может сильно помочь в автоматизации службы поддержки компании и др.
3. Изучение новой технологии

#### Цель

* Создание чат бота, поддерживающего беседу
* Изучение технологий, необходимых для этого

## Let's do this

#### Загрузим и подготовим данные

In [ ]:
                                                                                                                                                                                                            !wget https://www.dropbox.com/s/tfeozvw4hfnmufu/subtitles.txt

In [ ]:
!wget https://www.dropbox.com/s/cpcfs5y6z4qqhc0/Networks_weights.npz

In [ ]:
!wget https://www.dropbox.com/s/u9873fsq6r4he28/token_id.json

In [1]:
import re
import numpy as np
import codecs
import os
import json
from tqdm import tqdm

PAD_ix = -1

In [2]:
with open("dataset.txt") as fp:
    data = fp.read().split("\n")

In [3]:
data[:5]

['насколько\\@/ \\@/может\\@/ \\@/быть\\@/ \\@/силь\\@/ным\\@/ \\@/само\\@/вну\\@/ш\\@/ение?',
 'сила\\@/ \\@/вну\\@/шения\\@/ \\@/может\\@/ \\@/быть\\@/ \\@/очень\\@/ \\@/большо\\@/й.',
 'часто\\@/ \\@/ли\\@/ \\@/боле\\@/ют\\@/ \\@/поляр\\@/ники\\@/ \\@/на\\@/ \\@/рабо\\@/те?',
 'вообще\\@/ \\@/не\\@/ \\@/боле\\@/ют.',
 'где-нибудь\\@/ \\@/кроме\\@/ \\@/россии\\@/ \\@/еще\\@/ \\@/боятся\\@/ \\@/сквоз\\@/ня\\@/ка?']

In [4]:
def split(text):
    return re.split(r"\\@/", text)

In [5]:
phrase_list = []
for i in tqdm(range(0, len(data), 2)):
    try:
        phrase_list.append([split(data[i]), split(data[i+1])])
    except Exception:
        pass

100%|██████████| 235633/235633 [00:04<00:00, 52027.90it/s]


In [6]:
def join(l):
    return " ".join(l)

In [7]:
symb = ""
for s in tqdm(phrase_list):
    symb += " ".join(list(map(join, s))) + "  "

100%|██████████| 235632/235632 [00:01<00:00, 125276.03it/s]


Так как мы генирируем ответы посимвольно, определимся, какие символы у нас встречаются 

In [8]:
tokens = set(symb.split() + [" "])

tokens = list(tokens)

Для обозначения начала и конца будем использовать START и END. Превратим каждое наше предложение в список символов, которые оно содержит, добавим обозначение начала и конца

In [9]:
tokens = ["START"] + tokens + ["END"]

In [10]:
len(tokens)

15892

In [11]:
phrase_list = list(map(lambda s: list(map(lambda x: ["START"] + x + ["END"], s)), phrase_list))

In [12]:
len(phrase_list)

235632

Создадим словарь token - id

In [13]:
token_to_id = {tok: i for i, tok in enumerate(tokens)}

id_to_token = {token_to_id[tok]: tok for tok in token_to_id.keys()}

Будем превращать все наши предложения в матрицу, имеющую форму (кол-во предложений, мак-ое кол-во символов в предложении), если остаются пустые ячейки заполним их -1

In [14]:
def as_matrix(sequences,token_to_i, max_len=None,PAX_ix=PAD_ix, extra_max_len=None):
    max_len = max_len or max(map(len,sequences))
    max_len = np.min([max_len, extra_max_len]) if extra_max_len else max_len
    
    matrix = np.zeros((len(sequences),max_len),dtype='int32') -1
    for i,seq in enumerate(sequences):
        row_ix = list(filter(None.__ne__, map(token_to_i.get,seq)))[:max_len]
        matrix[i,:len(row_ix)] = row_ix
    
    return matrix

Преобразуем в токены

In [15]:
print(as_matrix(phrase_list[0], token_to_id))

[[    0  3528 11760 13068 11760  3197 11760 14661 15479 11760 14579 12208
   7599  3682 15891]
 [    0  2875 11760 12208 15730 11760 13068 11760  3197 11760  6054 11760
  14546  6578 15891]]


Сохраним

In [16]:
import json

In [17]:
with open("tokens_id.json", "w") as fp:
    json.dump({"token_to_id":token_to_id, "id_to_token": id_to_token, "tokens": tokens}, fp)

In [18]:
with open("tokens_id.json") as f:
    tokens_id = json.load(f)
token_to_id = tokens_id[u'token_to_id']
id_to_token = tokens_id["id_to_token"]
tokens = tokens_id["tokens"]
tokens = tokens

# Deep learning

Создадим сеть, которая будет получать на вход вопрос и генерировать посимвольно ответ на него

In [19]:
%env THEANO_FLAGS=device=gpu3,floatX=float32
import theano
import theano.tensor as T
import lasagne
from lasagne import layers
from lasagne.objectives import categorical_crossentropy
from lasagne.updates import adam
from hierarchical_softmax_layer import HierarchicalSoftmaxDenseLayer

env: THEANO_FLAGS=device=gpu3,floatX=float32


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 3: GeForce GTX 1080 (CNMeM is enabled with initial size: 22.0% of memory, cuDNN 5110)


In [20]:
input_sequence = T.matrix('token sequence','int32')
target_phonemes = T.matrix('target phonemes','int32')

In [21]:
##ENCODER
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)
l_mask = lasagne.layers.InputLayer(shape=(None, None),input_var=T.neq(input_sequence,-1))
l_emb = lasagne.layers.EmbeddingLayer(l_in, len(tokens), 256)
l_rnn = lasagne.layers.LSTMLayer(l_emb,256,mask_input=l_mask)
l_rnn = lasagne.layers.LSTMLayer(l_rnn,256,only_return_final=True,mask_input=l_mask)

##DECODER
transc_in = lasagne.layers.InputLayer(shape=(None, None),input_var=target_phonemes)
transc_mask = lasagne.layers.InputLayer(shape=(None, None),input_var=T.neq(target_phonemes,-1))
transc_emb = lasagne.layers.EmbeddingLayer(transc_in, len(tokens), 50)
transc_rnn = lasagne.layers.LSTMLayer(transc_emb,256,hid_init=l_rnn,mask_input=transc_mask)
transc_rnn = lasagne.layers.LSTMLayer(transc_rnn,256,hid_init=l_rnn,mask_input=transc_mask)


#flatten batch and time to be compatible with feedforward layers (will un-flatten later)
transc_rnn_flat = lasagne.layers.reshape(transc_rnn, (-1,transc_rnn.output_shape[-1]))

l_out = lasagne.layers.DenseLayer(transc_rnn_flat,len(tokens),nonlinearity=lasagne.nonlinearities.softmax)

In [22]:
weights = lasagne.layers.get_all_params(l_out, trainable=True)
print(weights)

[W, W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [23]:
network_output = lasagne.layers.get_output(l_out)
network_output = network_output.reshape([target_phonemes.shape[0], target_phonemes.shape[1], -1])

In [24]:
def crossentropy(answ):
    return -1*T.log(answ).mean()

In [25]:
predictions_flat = network_output[:,:-1,:].reshape([-1, len(tokens)])
targets = target_phonemes[:,1:].ravel()

mask = T.nonzero(T.neq(targets, -1))

loss = categorical_crossentropy(predictions_flat[mask], targets[mask]).mean()
updates = adam(loss, weights)

Компилируем

In [26]:

#training
train = theano.function([input_sequence, target_phonemes], loss, updates=updates, allow_input_downcast=True)

#computing loss without training
compute_cost = theano.function([input_sequence, target_phonemes], loss, allow_input_downcast=True)

## Создадим генератор ответов

In [27]:
#compile the function that computes probabilities for next token given previous text.

#reshape back into original shape
network_output = network_output.reshape((target_phonemes.shape[0],target_phonemes.shape[1],len(tokens)))
#predictions for next tokens (after sequence end)
last_word_probas = network_output[:,-1]
probs = theano.function([input_sequence,target_phonemes],last_word_probas,allow_input_downcast=True)


In [28]:
from apply_bpe import BPE

In [29]:
with open("codes.txt") as fp:
    bpe = BPE(fp)

In [30]:
bpe.segment("мы все учились понемногу")

'мы все учились по@@ не@@ мно@@ гу'

In [31]:
def apply_bpe(text):
    text = bpe.segment(text)
    text = re.sub(r"@@ ", "\@/", text)
    return re.sub(r" ", "\@/ \@/", text)

In [32]:
def generate_answer(question,answer_prefix = ("START",),t=1,sample=True, max_len=40):
    
    answer = list(answer_prefix)
    question = question.lower()
    question = ["START"] + re.split(r"\\@/", apply_bpe(question)) + ["END"]
    for _ in range(max_len):
#         print(as_matrix([question],token_to_id))
        if len(answer) < 2:
            answ_matrix = as_matrix([answer],token_to_id, max_len=2)
        else:
            answ_matrix = as_matrix([answer],token_to_id)
#         print(answ_matrix)
        next_let_probs = probs(as_matrix([question],token_to_id), answ_matrix).ravel()
        next_let_probs = next_let_probs**t / np.sum(next_let_probs**t)

        if sample:
            next_letter = np.random.choice(tokens,p=next_let_probs) 
        else:
            next_letter = tokens[np.argmax(next_let_probs)]
        
        answer.append(next_letter)

        if next_letter=="END":
            break
    return "".join(answer[1:-1])

In [33]:
print(generate_answer("насколько может быть сильным самовнушение?"))

себе.ленныйнынешнихджсрокалгоритсахараинстафилософсвоимфетиссыоперсекунаучно-балти①помидорами.ге.инфекособуспокопосмотретьрасскажпроизвелирайоненаличиичувствусвободуピтвоиэзоспособпищеблодневпеснива,


## Загрузим предобученые веса, если есть

In [ ]:
with np.load("Networks_weights.npz", encoding="bytes") as weights_file:
    lasagne.layers.set_all_param_values(l_out, weights_file["arr_0"])

### Тренируем

In [34]:
parametrs = {"token_to_i": token_to_id,
             "extra_max_len": 300}
def iterate_minibatches(source, batchsize, shuffle=True):
    source = np.array(source)
    if shuffle:
        indices = np.arange(len(source)-1)
        np.random.shuffle(indices)
    for start_idx in range(0, len(source) -1 - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = np.array(slice(start_idx, start_idx + batchsize))
#         print(len(source[excerpt]))
#         yield source[excerpt]
        yield as_matrix(source[excerpt][:, 0], **parametrs), as_matrix(source[excerpt][:, 1], **parametrs)

In [35]:
import config

In [36]:
telegram_token = config.telegram_token
telegram_chat_id = config.telegram_chat_id# id чата, в который будут отсылаться результаты в процессе обучения, так как она учится долго
dropbox_token = config.dropbox_token# Свежие веса загружаются на в облако, чтобы не потерять

In [37]:
from telebot import TeleBot
bot = TeleBot(telegram_token)

In [38]:
import dropbox

client = dropbox.client.DropboxClient(dropbox_token)

/anaconda2/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  app.launch_new_instance()


In [ ]:
train_message = "Epoch {epoch}:\nTime {time_:.2f} hours\nAverage loss: {avg_los:.5f}\nExample(насколько может быть сильным самовнушение?):\n\t{sample}"
train_error_messages = "Error on epoch when we train model: {}\n\t{}\nBatch:\n{}"

In [ ]:
from time import time, sleep
num_epoch = 100
batch_per_epoch = 9000
batch_size = 25
bot.send_message(145718567, "Learning is begin!")
error = False
for epoch in range(num_epoch):
#     try:
#     time.sleep(0.1)
    st = time()
    avg_cost = 0
    for batch_num, batch in enumerate(iterate_minibatches(phrase_list, batch_size)):
#         print(batch[0].shape, batch[1].shape)
#         print(batch[0].dtype, batch[1].dtype)
        try:
            avg_cost += train(batch[0], batch[1])
        except Exception as er:
            bot.send_message(telegram_chat_id, train_error_messages.format(epoch+1, er, (batch[0].shape, batch[1].shape)))
            error = True
            break

        if batch_num+1 == batch_per_epoch:
            break

    if error:
        break

    weights_file = "Networks_weights1.npz"
    np.savez(weights_file, layers.get_all_param_values(l_out))
#         try:
#             with open(weights_file, 'rb') as weights_file_dr:
#                 response = client.put_file('/' + weights_file, weights_file_dr)

#         except Exception as er:
#             bot.send_message(telegram_chat_id, "Error of download to Dropbox: {}".format(e))

    bot.send_message(145718567, train_message.format(time_=(time()-st)/60,
                                                     epoch=epoch+1,
                                                     avg_los=avg_cost/batch_per_epoch,
                                                    sample=generate_answer("насколько может быть сильным самовнушение?")))

#     except Exception as er:
#         bot.send_message(telegram_chat_id, "Error on epoch: {}\n\t{}".format(epoch+1, er))
#         break
bot.send_message(telegram_chat_id, "Learning is end!")

In [ ]:
generate_answer("насколько может быть сильным самовнушение?", sample=False)

In [ ]:
batch[0].shape, batch[1].shape

In [ ]:
del batch

In [ ]:
sorted